Introduction:

#Introduction to be written here

Import libraries

In [1]:
import numpy as np
import pandas as pd

worldCupMatches table

Reading worldCupMatches table

In [2]:
worldcup_matches = pd.read_csv('../data/WorldCupMatches.csv', encoding='utf-8')

Explore worldCupMatches table

In [3]:
worldcup_matches.head(10)

,Year,Datetime,Stage,Stadium,City,Home Team Name,Home Team Goals,Away Team Goals,Away Team Name,Win conditions,Attendance,Half-time Home Goals,Half-time Away Goals,Referee,Assistant 1,Assistant 2,RoundID,MatchID,Home Team Initials,Away Team Initials
0,1930.0,13 Jul 1930 - 15:00,Group 1,Pocitos,Montevideo,France,4.0,1.0,Mexico,,4444.0,3.0,0.0,LOMBARDI Domingo (URU),CRISTOPHE Henry (BEL),REGO Gilberto (BRA),201.0,1096.0,FRA,MEX
1,1930.0,13 Jul 1930 - 15:00,Group 4,Parque Central,Montevideo,USA,3.0,0.0,Belgium,,18346.0,2.0,0.0,MACIAS Jose (ARG),MATEUCCI Francisco (URU),WARNKEN Alberto (CHI),201.0,1090.0,USA,BEL
2,1930.0,14 Jul 1930 - 12:45,Group 2,Parque Central,Montevideo,Yugoslavia,2.0,1.0,Brazil,,24059.0,2.0,0.0,TEJADA Anibal (URU),VALLARINO Ricardo (URU),BALWAY Thomas (FRA),201.0,1093.0,YUG,BRA
3,1930.0,14 Jul 1930 - 14:50,Group 3,Pocitos,Montevideo,Romania,3.0,1.0,Peru,,2549.0,1.0,0.0,WARNKEN Alberto (CHI),LANGENUS Jean (BEL),MATEUCCI Francisco (URU),201.0,1098.0,ROU,PER
4,1930.0,15 Jul 1930 - 16:00,Group 1,Parque Central,Montevideo,Argentina,1.0,0.0,France,,23409.0,0.0,0.0,REGO Gilberto (BRA),SAUCEDO Ulises (BOL),RADULESCU Constantin (ROU),201.0,1085.0,ARG,FRA
5,1930.0,16 Jul 1930 - 14:45,Group 1,Parque Central,Montevideo,Chile,3.0,0.0,Mexico,,9249.0,1.0,0.0,CRISTOPHE Henry (BEL),APHESTEGUY Martin (URU),LANGENUS Jean (BEL),201.0,1095.0,CHI,MEX
6,1930.0,17 Jul 1930 - 12:45,Group 2,Parque Central,Montevideo,Yugoslavia,4.0,0.0,Bolivia,,18306.0,0.0,0.0,MATEUCCI Francisco (URU),LOMBARDI Domingo (URU),WARNKEN Alberto (CHI),201.0,1092.0,YUG,BOL
7,1930.0,17 Jul 1930 - 14:45,Group 4,Parque Central,Montevideo,USA,3.0,0.0,Paraguay,,18306.0,2.0,0.0,MACIAS Jose (ARG),APHESTEGUY Martin (URU),TEJADA Anibal (URU),201.0,1097.0,USA,PAR
8,1930.0,18 Jul 1930 - 14:30,Group 3,Estadio Centenario,Montevideo,Uruguay,1.0,0.0,Peru,,57735.0,0.0,0.0,LANGENUS Jean (BEL),BALWAY Thomas (FRA),CRISTOPHE Henry (BEL),201.0,1099.0,URU,PER
9,1930.0,19 Jul 1930 - 12:50,Group 1,Estadio Centenario,Montevideo,Chile,1.0,0.0,France,,2000.0,0.0,0.0,TEJADA Anibal (URU),LOMBARDI Domingo (URU),REGO Gilberto (BRA),201.0,1094.0,CHI,FRA


In [4]:
worldcup_matches.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4572 entries, 0 to 4571
Data columns (total 20 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   Year                  852 non-null    float64
 1   Datetime              852 non-null    object 
 2   Stage                 852 non-null    object 
 3   Stadium               852 non-null    object 
 4   City                  852 non-null    object 
 5   Home Team Name        852 non-null    object 
 6   Home Team Goals       852 non-null    float64
 7   Away Team Goals       852 non-null    float64
 8   Away Team Name        852 non-null    object 
 9   Win conditions        852 non-null    object 
 10  Attendance            850 non-null    float64
 11  Half-time Home Goals  852 non-null    float64
 12  Half-time Away Goals  852 non-null    float64
 13  Referee               852 non-null    object 
 14  Assistant 1           852 non-null    object 
 15  Assistant 2          

In [5]:
worldcup_matches.describe()

,Year,Home Team Goals,Away Team Goals,Attendance,Half-time Home Goals,Half-time Away Goals,RoundID,MatchID
count,852.000000,852.000000,852.000000,850.000000,852.000000,852.000000,8.520000e+02,8.520000e+02
mean,1985.089202,1.811033,1.022300,45164.800000,0.708920,0.428404,1.066177e+07,6.134687e+07
std,22.448825,1.610255,1.087573,23485.249247,0.937414,0.691252,2.729613e+07,1.110572e+08
min,1930.000000,0.000000,0.000000,2000.000000,0.000000,0.000000,2.010000e+02,2.500000e+01
25%,1970.000000,1.000000,0.000000,30000.000000,0.000000,0.000000,2.620000e+02,1.188750e+03
50%,1990.000000,2.000000,1.000000,41579.500000,0.000000,0.000000,3.370000e+02,2.191000e+03
75%,2002.000000,3.000000,2.000000,61374.500000,1.000000,1.000000,2.497220e+05,4.395006e+07
max,2014.000000,10.000000,7.000000,173850.000000,6.000000,5.000000,9.741060e+07,3.001865e+08


In [6]:
worldcup_matches.describe(include='object')

,Datetime,Stage,Stadium,City,Home Team Name,Away Team Name,Win conditions,Referee,Assistant 1,Assistant 2,Home Team Initials,Away Team Initials
count,852,852,852,852,852,852,852,852,852,852,852,852
unique,602,23,181,151,78,83,43,366,387,408,77,82
top,27 May 1934 - 16:30,Round of 16,Estadio Azteca,Mexico City,Brazil,Mexico,,Ravshan IRMATOV (UZB),ACHIK Redouane (MAR),KOCHKAROV Bakhadyr (KGZ),BRA,MEX
freq,8,72,19,23,82,38,787,10,7,10,82,38


In [7]:
worldcup_matches = worldcup_matches.rename(columns={'Win conditions': 'Win Conditions'})

In [8]:
worldcup_matches.isnull().sum()

Year                    3720
Datetime                3720
Stage                   3720
Stadium                 3720
City                    3720
Home Team Name          3720
Home Team Goals         3720
Away Team Goals         3720
Away Team Name          3720
Win Conditions          3720
Attendance              3722
Half-time Home Goals    3720
Half-time Away Goals    3720
Referee                 3720
Assistant 1             3720
Assistant 2             3720
RoundID                 3720
MatchID                 3720
Home Team Initials      3720
Away Team Initials      3720
dtype: int64

In [9]:
worldcup_matches.loc[worldcup_matches.Year.isnull()]

,Year,Datetime,Stage,Stadium,City,Home Team Name,Home Team Goals,Away Team Goals,Away Team Name,Win Conditions,Attendance,Half-time Home Goals,Half-time Away Goals,Referee,Assistant 1,Assistant 2,RoundID,MatchID,Home Team Initials,Away Team Initials
852,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
853,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
854,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
855,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
856,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4567,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4568,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4569,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4570,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [10]:
if worldcup_matches.loc[worldcup_matches.Year.isnull()].isnull().all().all():
    print("Where Year is Null, all cells are Null")
else:
    print("Where Year is Null, there are non-Null cells")

Where Year is Null, all cells are Null


In [11]:
worldcup_matches = worldcup_matches.dropna(subset=['Year'])

In [12]:
worldcup_matches.isnull().sum()

Year                    0
Datetime                0
Stage                   0
Stadium                 0
City                    0
Home Team Name          0
Home Team Goals         0
Away Team Goals         0
Away Team Name          0
Win Conditions          0
Attendance              2
Half-time Home Goals    0
Half-time Away Goals    0
Referee                 0
Assistant 1             0
Assistant 2             0
RoundID                 0
MatchID                 0
Home Team Initials      0
Away Team Initials      0
dtype: int64

In [13]:
worldcup_matches.loc[worldcup_matches.Attendance.isnull()]

,Year,Datetime,Stage,Stadium,City,Home Team Name,Home Team Goals,Away Team Goals,Away Team Name,Win Conditions,Attendance,Half-time Home Goals,Half-time Away Goals,Referee,Assistant 1,Assistant 2,RoundID,MatchID,Home Team Initials,Away Team Initials
823,2014.0,30 Jun 2014 - 17:00,Round of 16,Estadio Beira-Rio,Porto Alegre,Germany,2.0,1.0,Algeria,Germany win after extra time,NaN,0.0,0.0,RICCI Sandro (BRA),DE CARVALHO Emerson (BRA),VAN GASSE Marcelo (BRA),255951.0,300186460.0,GER,ALG
841,2014.0,30 Jun 2014 - 17:00,Round of 16,Estadio Beira-Rio,Porto Alegre,Germany,2.0,1.0,Algeria,Germany win after extra time,NaN,0.0,0.0,RICCI Sandro (BRA),DE CARVALHO Emerson (BRA),VAN GASSE Marcelo (BRA),255951.0,300186460.0,GER,ALG


In [14]:
worldcup_matches.duplicated().sum()

np.int64(16)

In [15]:
worldcup_matches.duplicated(subset=['MatchID']).sum()

np.int64(16)

In [16]:
worldcup_matches.loc[worldcup_matches.MatchID.duplicated(keep=False)]

,Year,Datetime,Stage,Stadium,City,Home Team Name,Home Team Goals,Away Team Goals,Away Team Name,Win Conditions,Attendance,Half-time Home Goals,Half-time Away Goals,Referee,Assistant 1,Assistant 2,RoundID,MatchID,Home Team Initials,Away Team Initials
820,2014.0,28 Jun 2014 - 13:00,Round of 16,Estadio Mineirao,Belo Horizonte,Brazil,1.0,1.0,Chile,Brazil win on penalties (3 - 2),57714.0,0.0,0.0,WEBB Howard (ENG),MULLARKEY Michael (ENG),Darren CANN (ENG),255951.0,300186487.0,BRA,CHI
821,2014.0,28 Jun 2014 - 17:00,Round of 16,Estadio do Maracana,Rio De Janeiro,Colombia,2.0,0.0,Uruguay,,73804.0,1.0,0.0,Bj�rn KUIPERS (NED),Sander VAN ROEKEL (NED),Erwin ZEINSTRA (NED),255951.0,300186491.0,COL,URU
822,2014.0,30 Jun 2014 - 13:00,Round of 16,Estadio Nacional,Brasilia,France,2.0,0.0,Nigeria,,67882.0,0.0,0.0,GEIGER Mark (USA),HURD Sean (USA),FLETCHER Joe (CAN),255951.0,300186462.0,FRA,NGA
823,2014.0,30 Jun 2014 - 17:00,Round of 16,Estadio Beira-Rio,Porto Alegre,Germany,2.0,1.0,Algeria,Germany win after extra time,NaN,0.0,0.0,RICCI Sandro (BRA),DE CARVALHO Emerson (BRA),VAN GASSE Marcelo (BRA),255951.0,300186460.0,GER,ALG
824,2014.0,04 Jul 2014 - 17:00,Quarter-finals,Estadio Castelao,Fortaleza,Brazil,2.0,1.0,Colombia,,60342.0,1.0,0.0,Carlos VELASCO CARBALLO (ESP),ALONSO FERNANDEZ Roberto (ESP),YUSTE Juan (ESP),255953.0,300186461.0,BRA,COL
825,2014.0,04 Jul 2014 - 13:00,Quarter-finals,Estadio do Maracana,Rio De Janeiro,France,0.0,1.0,Germany,,74240.0,0.0,1.0,PITANA Nestor (ARG),MAIDANA Hernan (ARG),BELATTI Juan Pablo (ARG),255953.0,300186485.0,FRA,GER
826,2014.0,08 Jul 2014 - 17:00,Semi-finals,Estadio Mineirao,Belo Horizonte,Brazil,1.0,7.0,Germany,,58141.0,0.0,5.0,RODRIGUEZ Marco (MEX),TORRENTERA Marvin (MEX),QUINTERO Marcos (MEX),255955.0,300186474.0,BRA,GER
827,2014.0,12 Jul 2014 - 17:00,Play-off for third place,Estadio Nacional,Brasilia,Brazil,0.0,3.0,Netherlands,,68034.0,0.0,2.0,HAIMOUDI Djamel (ALG),ACHIK Redouane (MAR),ETCHIALI Abdelhak (ALG),255957.0,300186502.0,BRA,NED
828,2014.0,13 Jul 2014 - 16:00,Final,Estadio do Maracana,Rio De Janeiro,Germany,1.0,0.0,Argentina,Germany win after extra time,74738.0,0.0,0.0,Nicola RIZZOLI (ITA),Renato FAVERANI (ITA),Andrea STEFANI (ITA),255959.0,300186501.0,GER,ARG
829,2014.0,09 Jul 2014 - 17:00,Semi-finals,Arena de Sao Paulo,Sao Paulo,Netherlands,0.0,0.0,Argentina,Argentina win on penalties (2 - 4),63267.0,0.0,0.0,C�neyt �AKIR (TUR),DURAN Bahattin (TUR),ONGUN Tarik (TUR),255955.0,300186490.0,NED,ARG


In [17]:
worldcup_matches = worldcup_matches.drop_duplicates(subset=['MatchID'])

In [18]:
worldcup_matches.duplicated().sum()

np.int64(0)

In [19]:
worldcup_matches.isnull().sum()

Year                    0
Datetime                0
Stage                   0
Stadium                 0
City                    0
Home Team Name          0
Home Team Goals         0
Away Team Goals         0
Away Team Name          0
Win Conditions          0
Attendance              1
Half-time Home Goals    0
Half-time Away Goals    0
Referee                 0
Assistant 1             0
Assistant 2             0
RoundID                 0
MatchID                 0
Home Team Initials      0
Away Team Initials      0
dtype: int64

In [20]:
worldcup_matches.loc[worldcup_matches.Attendance.isnull()]

,Year,Datetime,Stage,Stadium,City,Home Team Name,Home Team Goals,Away Team Goals,Away Team Name,Win Conditions,Attendance,Half-time Home Goals,Half-time Away Goals,Referee,Assistant 1,Assistant 2,RoundID,MatchID,Home Team Initials,Away Team Initials
823,2014.0,30 Jun 2014 - 17:00,Round of 16,Estadio Beira-Rio,Porto Alegre,Germany,2.0,1.0,Algeria,Germany win after extra time,NaN,0.0,0.0,RICCI Sandro (BRA),DE CARVALHO Emerson (BRA),VAN GASSE Marcelo (BRA),255951.0,300186460.0,GER,ALG


In [21]:
worldcup_matches.loc[worldcup_matches.Attendance.isnull(), 'Attendance'] = 43063

In [22]:
worldcup_matches.isnull().sum()

Year                    0
Datetime                0
Stage                   0
Stadium                 0
City                    0
Home Team Name          0
Home Team Goals         0
Away Team Goals         0
Away Team Name          0
Win Conditions          0
Attendance              0
Half-time Home Goals    0
Half-time Away Goals    0
Referee                 0
Assistant 1             0
Assistant 2             0
RoundID                 0
MatchID                 0
Home Team Initials      0
Away Team Initials      0
dtype: int64

In [23]:
worldcup_matches['Year'] = worldcup_matches['Year'].astype(np.uint16)
worldcup_matches['Home Team Goals'] = worldcup_matches['Home Team Goals'].astype(np.uint8)
worldcup_matches['Away Team Goals'] = worldcup_matches['Away Team Goals'].astype(np.uint8)
worldcup_matches['Attendance'] = worldcup_matches['Attendance'].astype(np.uint32)
worldcup_matches['Half-time Home Goals'] = worldcup_matches['Half-time Home Goals'].astype(np.uint8)
worldcup_matches['Half-time Away Goals'] = worldcup_matches['Half-time Away Goals'].astype(np.uint8)
worldcup_matches['RoundID'] = worldcup_matches['RoundID'].astype(np.uint32)
worldcup_matches['MatchID'] = worldcup_matches['MatchID'].astype(np.uint32)

In [24]:
worldcup_matches.dtypes

Year                    uint16
Datetime                object
Stage                   object
Stadium                 object
City                    object
Home Team Name          object
Home Team Goals          uint8
Away Team Goals          uint8
Away Team Name          object
Win Conditions          object
Attendance              uint32
Half-time Home Goals     uint8
Half-time Away Goals     uint8
Referee                 object
Assistant 1             object
Assistant 2             object
RoundID                 uint32
MatchID                 uint32
Home Team Initials      object
Away Team Initials      object
dtype: object

In [25]:
worldcup_matches.Datetime.unique()

array(['13 Jul 1930 - 15:00 ', '14 Jul 1930 - 12:45 ',
       '14 Jul 1930 - 14:50 ', '15 Jul 1930 - 16:00 ',
       '16 Jul 1930 - 14:45 ', '17 Jul 1930 - 12:45 ',
       '17 Jul 1930 - 14:45 ', '18 Jul 1930 - 14:30 ',
       '19 Jul 1930 - 12:50 ', '19 Jul 1930 - 15:00 ',
       '20 Jul 1930 - 13:00 ', '20 Jul 1930 - 15:00 ',
       '21 Jul 1930 - 14:50 ', '22 Jul 1930 - 14:45 ',
       '26 Jul 1930 - 14:45 ', '27 Jul 1930 - 14:45 ',
       '30 Jul 1930 - 14:15 ', '27 May 1934 - 16:30 ',
       '31 May 1934 - 16:30 ', '01 Jun 1934 - 16:30 ',
       '03 Jun 1934 - 16:30 ', '07 Jun 1934 - 18:00 ',
       '10 Jun 1934 - 17:30 ', '04 Jun 1938 - 17:00 ',
       '05 Jun 1938 - 17:00 ', '05 Jun 1938 - 17:30 ',
       '05 Jun 1938 - 18:30 ', '09 Jun 1938 - 18:00 ',
       '12 Jun 1938 - 17:00 ', '14 Jun 1938 - 18:00 ',
       '16 Jun 1938 - 18:00 ', '19 Jun 1938 - 17:00 ',
       '24 Jun 1950 - 15:00 ', '25 Jun 1950 - 15:00 ',
       '28 Jun 1950 - 15:00 ', '29 Jun 1950 - 15:00 ',
       '29

In [26]:
months_abb = {
    'January': 'Jan',
    'February': 'Feb',
    'March': 'Mar',
    'April': 'Apr',
    'May': 'May',
    'June': 'Jun',
    'July': 'Jul',
    'August': 'Aug',
    'September': 'Sep',
    'October': 'Oct',
    'November': 'Nov',
    'December': 'Dec',
}
months_pattern = '|'.join(months_abb.keys())
worldcup_matches.Datetime = worldcup_matches.Datetime.str.replace(months_pattern, lambda x: months_abb[x.group()], regex=True).str.strip()
worldcup_matches.Datetime.unique()

array(['13 Jul 1930 - 15:00', '14 Jul 1930 - 12:45',
       '14 Jul 1930 - 14:50', '15 Jul 1930 - 16:00',
       '16 Jul 1930 - 14:45', '17 Jul 1930 - 12:45',
       '17 Jul 1930 - 14:45', '18 Jul 1930 - 14:30',
       '19 Jul 1930 - 12:50', '19 Jul 1930 - 15:00',
       '20 Jul 1930 - 13:00', '20 Jul 1930 - 15:00',
       '21 Jul 1930 - 14:50', '22 Jul 1930 - 14:45',
       '26 Jul 1930 - 14:45', '27 Jul 1930 - 14:45',
       '30 Jul 1930 - 14:15', '27 May 1934 - 16:30',
       '31 May 1934 - 16:30', '01 Jun 1934 - 16:30',
       '03 Jun 1934 - 16:30', '07 Jun 1934 - 18:00',
       '10 Jun 1934 - 17:30', '04 Jun 1938 - 17:00',
       '05 Jun 1938 - 17:00', '05 Jun 1938 - 17:30',
       '05 Jun 1938 - 18:30', '09 Jun 1938 - 18:00',
       '12 Jun 1938 - 17:00', '14 Jun 1938 - 18:00',
       '16 Jun 1938 - 18:00', '19 Jun 1938 - 17:00',
       '24 Jun 1950 - 15:00', '25 Jun 1950 - 15:00',
       '28 Jun 1950 - 15:00', '29 Jun 1950 - 15:00',
       '29 Jun 1950 - 15:30', '01 Jul 1950 - 1

In [27]:
worldcup_matches['Datetime'] = pd.to_datetime(worldcup_matches['Datetime'], format='%d %b %Y - %H:%M')
worldcup_matches.dtypes

Year                            uint16
Datetime                datetime64[ns]
Stage                           object
Stadium                         object
City                            object
Home Team Name                  object
Home Team Goals                  uint8
Away Team Goals                  uint8
Away Team Name                  object
Win Conditions                  object
Attendance                      uint32
Half-time Home Goals             uint8
Half-time Away Goals             uint8
Referee                         object
Assistant 1                     object
Assistant 2                     object
RoundID                         uint32
MatchID                         uint32
Home Team Initials              object
Away Team Initials              object
dtype: object

In [28]:
(worldcup_matches.Datetime.dt.year != worldcup_matches.Year).sum()

np.int64(0)

In [29]:
# for col in worldcup_matches.columns:
    # print(f"Unique values in column {col}: {(worldcup_matches[col].unique())}")

In [30]:
worldcup_matches.Stage.unique()

array(['Group 1', 'Group 4', 'Group 2', 'Group 3', 'Semi-finals', 'Final',
       'Preliminary round', 'Quarter-finals', 'Match for third place',
       'First round', 'Group 6', 'Group B', 'Group A', 'Group 5',
       'Group C', 'Group D', 'Group F', 'Group E', 'Round of 16',
       'Group H', 'Group G', 'Third place', 'Play-off for third place'],
      dtype=object)

In [31]:
rounds = {
    'Group 1': 'Group A',
    'Group 2': 'Group B',
    'Group 3': 'Group C',
    'Group 4': 'Group D',
    'Group 5': 'Group E',
    'Group 6': 'Group F',
    'Match for third place': 'Third place',
    'Play-off for third place': 'Third place',
    'Preliminary round': 'Round of 16',
    'First round': 'Round of 16',
}

worldcup_matches = worldcup_matches.replace(rounds)

In [32]:
worldcup_matches.Stage.unique()

array(['Group A', 'Group D', 'Group B', 'Group C', 'Semi-finals', 'Final',
       'Round of 16', 'Quarter-finals', 'Third place', 'Group F',
       'Group E', 'Group H', 'Group G'], dtype=object)

In [33]:
worldcup_matches.Stadium.unique()

array(['Pocitos', 'Parque Central', 'Estadio Centenario',
       'Stadio Benito Mussolini', 'Giorgio Ascarelli', 'San Siro',
       'Littorale', 'Giovanni Berta', 'Luigi Ferraris', 'Nazionale PNF',
       'Littorio', 'Parc des Princes', 'Velodrome Municipale',
       'Stade Olympique', 'Stade Municipal', 'Stade V�lodrome',
       'Stade de la Meinau', 'Cavee Verte', 'Stade du Parc Lescure',
       'Victor Boucquey', 'Fort Carree',
       'Maracan� - Est�dio Jornalista M�rio Filho', 'Durival de Brito',
       'Pacaembu', 'Independencia', 'Eucaliptos', 'Ilha do Retiro',
       'Wankdorf Stadium', 'Hardturm', 'Charmilles', 'La Pontaise',
       'St. Jakob', 'Comunale di Cornaredo', 'Rasunda Stadium',
       'Nya Ullevi', 'Malmo Stadion', 'Jarnvallen', 'Idrottsparken',
       'Arosvallen', 'Rimnersvallen', 'Orjans Vall', 'Ryavallen',
       'Olympia Stadium', 'Tunavallen', 'Eyravallen', 'Carlos Dittborn',
       'Estadio Sausalito', 'Estadio El Teniente-Codelco', 'Nacional',
       'Wemble

In [34]:
stadium_fixes = {
    'Stade V�lodrome': 'Stade Vélodrome',
    'Maracan� - Est�dio Jornalista M�rio Filho': 'Maracanã - Estádio Jornalista Mário Filho',
    'Nou Camp - Estadio Le�n': 'Nou Camp - Estadio León',
    'Estadio Jos� Mar�a Minella': 'Estadio José María Minella',
    'Estadio Ol�mpico Chateau Carreras': 'Estadio Olímpico Chateau Carreras',
    'Estadio Municipal de Bala�dos': 'Estadio Municipal de Balaídos',
    'Estadio Ol�mpico Universitario': 'Estadio Olímpico Universitario'
}

worldcup_matches['Stadium'] = worldcup_matches['Stadium'].replace(stadium_fixes)

In [35]:
worldcup_matches.Stadium.unique()

array(['Pocitos', 'Parque Central', 'Estadio Centenario',
       'Stadio Benito Mussolini', 'Giorgio Ascarelli', 'San Siro',
       'Littorale', 'Giovanni Berta', 'Luigi Ferraris', 'Nazionale PNF',
       'Littorio', 'Parc des Princes', 'Velodrome Municipale',
       'Stade Olympique', 'Stade Municipal', 'Stade Vélodrome',
       'Stade de la Meinau', 'Cavee Verte', 'Stade du Parc Lescure',
       'Victor Boucquey', 'Fort Carree',
       'Maracanã - Estádio Jornalista Mário Filho', 'Durival de Brito',
       'Pacaembu', 'Independencia', 'Eucaliptos', 'Ilha do Retiro',
       'Wankdorf Stadium', 'Hardturm', 'Charmilles', 'La Pontaise',
       'St. Jakob', 'Comunale di Cornaredo', 'Rasunda Stadium',
       'Nya Ullevi', 'Malmo Stadion', 'Jarnvallen', 'Idrottsparken',
       'Arosvallen', 'Rimnersvallen', 'Orjans Vall', 'Ryavallen',
       'Olympia Stadium', 'Tunavallen', 'Eyravallen', 'Carlos Dittborn',
       'Estadio Sausalito', 'Estadio El Teniente-Codelco', 'Nacional',
       'Wemble

In [36]:
worldcup_matches.City.unique()

array(['Montevideo ', 'Turin ', 'Naples ', 'Milan ', 'Bologna ',
       'Florence ', 'Genoa ', 'Rome ', 'Trieste ', 'Paris ', 'Reims ',
       'Colombes ', 'Toulouse ', 'Marseilles ', 'Strasbourg ',
       'Le Havre ', 'Bordeaux ', 'Lille ', 'Antibes ', 'Rio De Janeiro ',
       'Curitiba ', 'Sao Paulo ', 'Belo Horizonte ', 'Porto Alegre ',
       'Recife ', 'Berne ', 'Zurich ', 'Geneva ', 'Lausanne ', 'Basel ',
       'Lugano ', 'Solna ', 'Gothenburg ', 'Malm� ', 'Sandviken ',
       'Norrk�Ping ', 'Vasteras ', 'Udevalla ', 'Halmstad ', 'Boras ',
       'Helsingborg ', 'Eskilstuna ', 'Orebro ', 'Arica ',
       'Vina Del Mar ', 'Rancagua ', 'Santiago De Chile ', 'London ',
       'Sheffield ', 'Liverpool ', 'Middlesbrough ', 'Manchester ',
       'Birmingham ', 'Sunderland ', 'Mexico City ', 'Puebla ', 'Leon ',
       'Guadalajara ', 'Toluca ', 'Frankfurt/Main ', 'Berlin West ',
       'Hamburg ', 'Dortmund ', 'Hanover ', 'D�Sseldorf ', 'Munich ',
       'Stuttgart ', 'Gelsenkirchen '

In [37]:
city_fixes = {
    'Malm� ': 'Malmö',
    'Norrk�Ping ': 'Norrköping',
    'D�Sseldorf ': 'Düsseldorf',
    'La Coru�A ': 'La Coruña'
}

worldcup_matches['City'] = worldcup_matches['City'].replace(city_fixes)

In [38]:
worldcup_matches.City.unique()

array(['Montevideo ', 'Turin ', 'Naples ', 'Milan ', 'Bologna ',
       'Florence ', 'Genoa ', 'Rome ', 'Trieste ', 'Paris ', 'Reims ',
       'Colombes ', 'Toulouse ', 'Marseilles ', 'Strasbourg ',
       'Le Havre ', 'Bordeaux ', 'Lille ', 'Antibes ', 'Rio De Janeiro ',
       'Curitiba ', 'Sao Paulo ', 'Belo Horizonte ', 'Porto Alegre ',
       'Recife ', 'Berne ', 'Zurich ', 'Geneva ', 'Lausanne ', 'Basel ',
       'Lugano ', 'Solna ', 'Gothenburg ', 'Malmö', 'Sandviken ',
       'Norrköping', 'Vasteras ', 'Udevalla ', 'Halmstad ', 'Boras ',
       'Helsingborg ', 'Eskilstuna ', 'Orebro ', 'Arica ',
       'Vina Del Mar ', 'Rancagua ', 'Santiago De Chile ', 'London ',
       'Sheffield ', 'Liverpool ', 'Middlesbrough ', 'Manchester ',
       'Birmingham ', 'Sunderland ', 'Mexico City ', 'Puebla ', 'Leon ',
       'Guadalajara ', 'Toluca ', 'Frankfurt/Main ', 'Berlin West ',
       'Hamburg ', 'Dortmund ', 'Hanover ', 'Düsseldorf', 'Munich ',
       'Stuttgart ', 'Gelsenkirchen ', '

In [39]:
pd.concat([worldcup_matches['Home Team Name'], worldcup_matches['Away Team Name']]).unique()

array(['France', 'USA', 'Yugoslavia', 'Romania', 'Argentina', 'Chile',
       'Uruguay', 'Brazil', 'Paraguay', 'Austria', 'Hungary',
       'Switzerland', 'Sweden', 'Germany', 'Spain', 'Italy',
       'Czechoslovakia', 'Cuba', 'England', 'Germany FR', 'Turkey',
       'Soviet Union', 'Northern Ireland', 'Mexico', 'Wales', 'Portugal',
       'Korea DPR', 'Peru', 'Belgium', 'Bulgaria', 'German DR', 'Zaire',
       'Poland', 'Australia', 'Scotland', 'Netherlands', 'Haiti',
       'Tunisia', 'Algeria', 'Honduras', 'Canada', 'Morocco',
       'Korea Republic', 'Iraq', 'Denmark', 'rn">United Arab Emirates',
       'Costa Rica', 'Cameroon', 'rn">Republic of Ireland', 'Colombia',
       'Norway', 'Nigeria', 'Saudi Arabia', 'Bolivia', 'Russia', 'Greece',
       'Jamaica', 'South Africa', 'Japan', 'Croatia', 'China PR',
       'Senegal', 'Slovenia', 'Ecuador', 'rn">Trinidad and Tobago',
       'rn">Serbia and Montenegro', 'Angola', 'Czech Republic', 'Togo',
       'Iran', "C�te d'Ivoire", 'Ghana

In [40]:
countries = {
    "C�te d'Ivoire": "Côte d'Ivoire",
    'Germany FR': 'Germany',
    'German DR': 'Germany',
    'Soviet Union': 'Russia',
    'Czechoslovakia': 'Czech Republic',
    'rn">United Arab Emirates': 'United Arab Emirates',
    'rn">Republic of Ireland': 'Republic of Ireland',
    'rn">Trinidad and Tobago': 'Trinidad and Tobago',
    'rn">Serbia and Montenegro': 'Serbia',
    'Yugoslavia': 'Serbia',
    'rn">Bosnia and Herzegovina': 'Bosnia and Herzegovina',
    'Dutch East Indies': 'Indonesia',
    'Zaire': 'Congo DR',
    'Iran': 'IR Iran',
}

countries_pattern = '|'.join(countries.keys())
worldcup_matches['Home Team Name'] = worldcup_matches['Home Team Name'].str.replace(countries_pattern, lambda x: countries[x.group()], regex=True).str.strip()
worldcup_matches['Away Team Name'] = worldcup_matches['Away Team Name'].str.replace(countries_pattern, lambda x: countries[x.group()], regex=True).str.strip()
worldcup_matches['Win Conditions'] = worldcup_matches['Win Conditions'].str.replace(countries_pattern, lambda x: countries[x.group()], regex=True).str.strip()

In [41]:
teams_names = pd.concat([worldcup_matches['Home Team Name'], worldcup_matches['Away Team Name']]).unique()
teams_names

array(['France', 'USA', 'Serbia', 'Romania', 'Argentina', 'Chile',
       'Uruguay', 'Brazil', 'Paraguay', 'Austria', 'Hungary',
       'Switzerland', 'Sweden', 'Germany', 'Spain', 'Italy',
       'Czech Republic', 'Cuba', 'England', 'Turkey', 'Russia',
       'Northern Ireland', 'Mexico', 'Wales', 'Portugal', 'Korea DPR',
       'Peru', 'Belgium', 'Bulgaria', 'Congo DR', 'Poland', 'Australia',
       'Scotland', 'Netherlands', 'Haiti', 'Tunisia', 'Algeria',
       'Honduras', 'Canada', 'Morocco', 'Korea Republic', 'Iraq',
       'Denmark', 'United Arab Emirates', 'Costa Rica', 'Cameroon',
       'Republic of Ireland', 'Colombia', 'Norway', 'Nigeria',
       'Saudi Arabia', 'Bolivia', 'Greece', 'Jamaica', 'South Africa',
       'Japan', 'Croatia', 'China PR', 'Senegal', 'Slovenia', 'Ecuador',
       'Trinidad and Tobago', 'Angola', 'Togo', 'IR Iran',
       "Côte d'Ivoire", 'Ghana', 'Ukraine', 'New Zealand', 'Slovakia',
       'IR IR Iran', 'Bosnia and Herzegovina', 'Egypt', 'Indonesia

In [42]:
worldcup_matches[worldcup_matches['Home Team Goals']==worldcup_matches['Home Team Goals'].max()]

,Year,Datetime,Stage,Stadium,City,Home Team Name,Home Team Goals,Away Team Goals,Away Team Name,Win Conditions,Attendance,Half-time Home Goals,Half-time Away Goals,Referee,Assistant 1,Assistant 2,RoundID,MatchID,Home Team Initials,Away Team Initials
312,1982,1982-06-15 21:00:00,Group C,Nuevo Estadio,Elche,Hungary,10,1,El Salvador,,23000,3,0,AL DOY Ebrahim (BHR),CORVER Charles (NED),LUND-SORENSEN Henning (DEN),293,896,HUN,SLV


In [43]:
worldcup_matches[worldcup_matches['Away Team Goals']==worldcup_matches['Away Team Goals'].max()]

,Year,Datetime,Stage,Stadium,City,Home Team Name,Home Team Goals,Away Team Goals,Away Team Name,Win Conditions,Attendance,Half-time Home Goals,Half-time Away Goals,Referee,Assistant 1,Assistant 2,RoundID,MatchID,Home Team Initials,Away Team Initials
246,1974,1974-06-19 19:30:00,Group D,Olympiastadion,Munich,Haiti,0,7,Poland,,25300,0,5,SUPPIAH George (SIN),BIWERSI Ferdinand (GER),ESCHWEILER Walter (GER),262,2085,HAI,POL
826,2014,2014-07-08 17:00:00,Semi-finals,Estadio Mineirao,Belo Horizonte,Brazil,1,7,Germany,,58141,0,5,RODRIGUEZ Marco (MEX),TORRENTERA Marvin (MEX),QUINTERO Marcos (MEX),255955,300186474,BRA,GER


In [44]:
initials = {
    'Germany': 'GER',
    'Russia': 'RUS',
    'Czech Republic': 'CZE',
    'Serbia': 'SRB',
    'Indonesia': 'IND',
    'Congo DR': 'COD',
}
worldcup_matches['Home Team Initials'] = worldcup_matches['Home Team Initials'].replace(initials)
worldcup_matches['Away Team Initials'] = worldcup_matches['Away Team Initials'].replace(initials)

In [45]:
worldcup_matches['Win Conditions'].unique()

array(['', 'Austria win after extra time', 'Italy win after extra time',
       'Brazil win after extra time',
       'Czech Republic win after extra time',
       'Hungary win after extra time',
       'Northern Ireland win after extra time',
       'England win after extra time', 'Germany win after extra time',
       'Uruguay win after extra time', 'Argentina win after extra time',
       'win on penalties (5 - 4)', 'Belgium win after extra time',
       'France win on penalties (3 - 4)',
       'Germany win on penalties (4 - 1)',
       'Belgium win on penalties (4 - 5)', 'France win after extra time',
       'Cameroon win after extra time',
       'Republic of Ireland win on penalties (5 - 4)',
       'Serbia win after extra time', 'win on penalties (2 - 3)',
       'win on penalties (3 - 4)', 'win on penalties (4 - 3)',
       'Bulgaria win on penalties (1 - 3)',
       'Sweden win on penalties (4 - 5)',
       'Brazil win on penalties (3 - 2)',
       'Argentina win on penalties

In [46]:
worldcup_matches.loc[worldcup_matches['Win Conditions'].str.startswith('win'), :]

,Year,Datetime,Stage,Stadium,City,Home Team Name,Home Team Goals,Away Team Goals,Away Team Name,Win Conditions,Attendance,Half-time Home Goals,Half-time Away Goals,Referee,Assistant 1,Assistant 2,RoundID,MatchID,Home Team Initials,Away Team Initials
357,1982,1982-07-08 21:00:00,Semi-finals,Ramon Sanchez Pizjuan,Seville,Germany,3,3,France,win on penalties (5 - 4),70000,0,0,CORVER Charles (NED),GALLER Bruno (SUI),VALENTINE Robert (SCO),295,914,FRG,FRA
456,1990,1990-06-30 17:00:00,Quarter-finals,Comunale,Florence,Serbia,0,0,Argentina,win on penalties (2 - 3),38971,0,0,ROETHLISBERGER Kurt (SUI),JOUINI Neji (TUN),HANSAL Mohamed (ALG),751,31,YUG,ARG
460,1990,1990-07-03 20:00:00,Semi-finals,San Paolo,Naples,Italy,1,1,Argentina,win on penalties (3 - 4),59978,0,0,VAUTROT Michel (FRA),LISTKIEWICZ Michal (POL),MIKKELSEN Peter (DEN),3464,28,ITA,ARG
461,1990,1990-07-04 20:00:00,Semi-finals,Stadio delle Alpi,Turin,Germany,1,1,England,win on penalties (4 - 3),62628,0,0,RAMIZ WRIGHT Jose (BRA),QUINIOU Joel (FRA),PEREZ HOYOS Armando (COL),3464,159,FRG,ENG


In [47]:
"""
mask = worldcup_matches['Win Conditions'].str.startswith('win', na=False)

worldcup_matches.loc[mask, 'home_pen_score'] = worldcup_matches.loc[mask, 'Win Conditions'].str[-6].astype(np.uint8)
worldcup_matches.loc[mask, 'away_pen_score'] = worldcup_matches.loc[mask, 'Win Conditions'].str[-2].astype(np.uint8)

worldcup_matches.loc[mask, 'winner'] = np.where(
    worldcup_matches.loc[mask, 'home_pen_score'] > worldcup_matches.loc[mask, 'away_pen_score'],
    worldcup_matches.loc[mask, 'Home Team Name'],
    worldcup_matches.loc[mask, 'Away Team Name']
    )
worldcup_matches.loc[mask, 'Win Conditions'] = worldcup_matches.loc[mask, 'winner'] + ' ' + worldcup_matches['Win Conditions']
worldcup_matches.drop(['home_pen_score', 'away_pen_score', 'winner'], axis=1, inplace=True)
"""

"\nmask = worldcup_matches['Win Conditions'].str.startswith('win', na=False)\n\nworldcup_matches.loc[mask, 'home_pen_score'] = worldcup_matches.loc[mask, 'Win Conditions'].str[-6].astype(np.uint8)\nworldcup_matches.loc[mask, 'away_pen_score'] = worldcup_matches.loc[mask, 'Win Conditions'].str[-2].astype(np.uint8)\n\nworldcup_matches.loc[mask, 'winner'] = np.where(\n    worldcup_matches.loc[mask, 'home_pen_score'] > worldcup_matches.loc[mask, 'away_pen_score'],\n    worldcup_matches.loc[mask, 'Home Team Name'],\n    worldcup_matches.loc[mask, 'Away Team Name']\n    )\nworldcup_matches.loc[mask, 'Win Conditions'] = worldcup_matches.loc[mask, 'winner'] + ' ' + worldcup_matches['Win Conditions']\nworldcup_matches.drop(['home_pen_score', 'away_pen_score', 'winner'], axis=1, inplace=True)\n"

In [48]:
# worldcup_matches['Win Conditions'].unique()

In [51]:
mask_draw = (worldcup_matches['Home Team Goals'] == worldcup_matches['Away Team Goals']) & (~worldcup_matches['Win Conditions'].str.contains('penalties', na=False))
mask_penalties = (worldcup_matches['Home Team Goals'] == worldcup_matches['Away Team Goals']) & (worldcup_matches['Win Conditions'].str.contains('penalties', na=False))
mask_win = worldcup_matches['Home Team Goals'] != worldcup_matches['Away Team Goals']
mask_non_draw = ~mask_draw

worldcup_matches.loc[mask_penalties, 'home_pen_score'] = worldcup_matches.loc[mask_penalties, 'Win Conditions'].str[-6].astype(np.uint8)
worldcup_matches.loc[mask_penalties, 'away_pen_score'] = worldcup_matches.loc[mask_penalties, 'Win Conditions'].str[-2].astype(np.uint8)

worldcup_matches.loc[mask_draw, 'winner'] = 'Draw'

worldcup_matches.loc[mask_penalties, 'winner'] = np.where(
    worldcup_matches.loc[mask_penalties, 'home_pen_score'] > worldcup_matches.loc[mask_penalties, 'away_pen_score'],
    worldcup_matches.loc[mask_penalties, 'Home Team Name'],
    worldcup_matches.loc[mask_penalties, 'Away Team Name']
)

worldcup_matches.loc[mask_win, 'winner'] = np.where(
    worldcup_matches.loc[mask_win, 'Home Team Goals'] < worldcup_matches.loc[mask_win, 'Away Team Goals'],
    worldcup_matches.loc[mask_win, 'Away Team Name'],
    worldcup_matches.loc[mask_win, 'Home Team Name']
)

worldcup_matches.loc[mask_draw, 'Win Conditions'] = worldcup_matches.loc[mask_draw, 'winner']

teams_pattern = '|'.join(teams_names)
mask_teams = mask_non_draw & worldcup_matches.loc[mask_non_draw, 'Win Conditions'].str.contains(teams_pattern, case=False, na=False, regex=True)

if (worldcup_matches.loc[mask_teams, 'winner'] == worldcup_matches.loc[mask_teams, 'Win Conditions'].str.split(' ').str[0]).all():
    print("All marked teams as winning are correct")
else:
    print("Teams winning difference")
    print(f"{worldcup_matches.loc[mask_teams, ['Home Team Name', 'Home Team Goals','Away Team Goals', 'Away Team Name', 'Win Conditions']]}")
    print(f"{worldcup_matches.loc[mask_teams, ['Home Team Name', 'Home Team Goals','Away Team Goals', 'Away Team Name', 'Win Conditions']]}")
    win_cond = worldcup_matches.loc[mask_teams, 'Win Conditions'].str.split(' ', n=1).str[1]
    worldcup_matches.loc[mask_teams, 'Win Conditions'] = worldcup_matches.loc[mask_teams, 'winner'] + ' ' + win_cond

mask_unk_win = worldcup_matches['Win Conditions'].str.startswith('win', na=False)
worldcup_matches.loc[mask_unk_win, 'Win Conditions'] = worldcup_matches.loc[mask_unk_win, 'winner'] + ' ' + worldcup_matches.loc[mask_unk_win, 'Win Conditions']

mask_unk_res = worldcup_matches['Win Conditions'] == ''
worldcup_matches.loc[mask_unk_res, 'Win Conditions'] = worldcup_matches.loc[mask_unk_res, 'winner'] + ' win'

worldcup_matches.drop(['home_pen_score', 'away_pen_score', 'winner'], axis=1, inplace=True)

ValueError: The truth value of a Series is ambiguous. Use a.empty, a.bool(), a.item(), a.any() or a.all().

In [50]:
# worldcup_matches['Attendance'].plot(kind='hist')
# worldcup_matches['Home Team Goals'].plot(kind='hist')
# worldcup_matches['Away Team Goals'].plot(kind='hist')